Load in Packages:

In [ ]:
# core imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn core
from sklearn.model_selection import (train_test_split, StratifiedKFold, GridSearchCV)
from sklearn.preprocessing import (OneHotEncoder, StandardScaler, FunctionTransformer)
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay)
from scipy import sparse
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from joblib import Memory
from sklearn.base import clone

Load in Data:

In [ ]:
train_df = pd.read_csv("project_adult.csv")
validation_df = pd.read_csv("project_validation_inputs.csv")

**PREPROCESSING:**

Perform basic data cleaning:

In [ ]:
# NAME THE FIRST COLUMN (THIS IS THE INDEX COLUMN)

train_df = train_df.rename(columns={"Unnamed: 0": "index"})
validation_df = validation_df.rename(columns={"Unnamed: 0": "index"})

# CLEAN THE DATA:
def basic_clean(df):
    # standardize column names
    df.columns = (
        df.columns
            .str.strip() # remove leading/trailing spaces
            .str.lower() # convert to lowercase
            .str.replace(" ", "_") # replace spaces with underscore
            .str.replace("-", "_") # replace hyphens with underscore
    )
    df = df.replace("?", np.nan) # replace "?" with NaN within data

    # clean data within columns
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.lower()# convert to lowercase
        df[col] = df[col].str.strip() # remove leading/trailing spaces
        df[col] = df[col].str.replace("-", "_") # replace hyphens with underscore
    return df

# APPLY THE CLEANING FUNCTION:
train_df = basic_clean(train_df)
validation_df = basic_clean(validation_df)

# normalize income labels:
train_df["income"] = train_df["income"].str.strip()

# adjust income values to be binary
    # <=50K -> 0  (negative class)
    #  >50K -> 1  (positive class)
train_df["income"] = train_df["income"].replace({
    "<=50k": 0, "<=50k.": 0,
    ">50k": 1,  ">50k.": 1
})

In [ ]:
# check results
print(train_df["income"].value_counts(dropna=False))

EDA:

In [ ]:
# SHAPE
print("Train shape:", train_df.shape)
print("Validation shape:", validation_df.shape)

In [ ]:
# print number of NAs in training data
print("\nMissing count per variable in training data:")
print(train_df.isna().sum())

In [ ]:
# print number of NAs in validation data
print("\nMissing count per variable in validation data:")
print(validation_df.isna().sum())

In [ ]:
# count each class and normalize to percentage
income_counts = train_df["income"].value_counts(normalize=True) * 100

print(income_counts)

In [ ]:
# calculate % earning >50K per hours per week
hours_income = (
    train_df.groupby(["hours_per_week"])["income"]
            .mean()
            .reset_index()
)

# convert proportions to percentages
hours_income["income"] = hours_income["income"] * 100

# plot
plt.figure(figsize=(10,6))
plt.barh(hours_income["hours_per_week"], hours_income["income"], color="skyblue")

plt.xlabel("% earning >50K")
plt.ylabel("Hrs Worked Per Week")
plt.title("Percentage of Individuals Earning >50K by Hrs Worked Per Week")
plt.show()

In [ ]:
# calculate % earning >50K per education level
edu_income = (
    train_df.groupby(["education", "education_num"])["income"]
            .mean()
            .reset_index()
            .sort_values("education_num")  # sort by numeric education level
)

# convert proportions to percentages
edu_income["income"] = edu_income["income"] * 100

# plot
plt.figure(figsize=(10,6))
plt.barh(edu_income["education"], edu_income["income"], color="skyblue")

plt.xlabel("% earning >50K")
plt.ylabel("Education Level")
plt.title("Percentage of Individuals Earning >50K by Education Level (Sorted by Education Num)")
plt.show()

In [ ]:
# explore prevalance of age groups:

# group ages by decade 
age_counts_by_decade = (
    train_df.groupby((train_df["age"] // 10) * 10)  # integer division → 10s, 20s, 30s, etc.
            .size()
            .reset_index(name="count")
            .rename(columns={"age": "age_decade"})  # rename column for clarity
            .sort_values("age_decade")
)

print(age_counts_by_decade.to_string(index=False))

In [ ]:
# show each level of education, the number associated with it, and count of each for better understanding
edu_counts = (
    train_df.groupby(["education_num", "education"])
            .size()                               # counts
            .reset_index(name="count")            # put into a column
            .sort_values("education_num")         # orders by the numeric code
)

#print
print(edu_counts.to_string(index=False))

Encoding categorical values:

In [ ]:
# separate target from features (TRAIN ONLY)
y_train = train_df["income"] # target
X_train = train_df.drop(columns=["income"])

# stratified split 
X_tr, X_te, y_tr, y_te = train_test_split(
    X_train, y_train, 
    test_size = 0.20, 
    stratify = y_train, 
    random_state = 42
    )

In [ ]:
# IDENTIFY COLUMN TYPES: 
categorical_cols = X_train.select_dtypes(include=["object"]).columns
numerical_cols = X_train.select_dtypes(include=["int64", "float64"]).columns

# print
print("Categorical columns:", categorical_cols.tolist())
print("Numerical columns:", numerical_cols.tolist())

In [ ]:
# DEFINE TRANSFORMERS (processing steps):
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), # fill NaN with most common
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")), # fill NaN with median
    ("scaler", StandardScaler()) # standardize
])

In [ ]:
# COMBINE INTO PREPROCESSER
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols)
    ]
)

In [ ]:
# FIT ON TRAINING, TRANSFORM BOTH
X_train_processed = preprocessor.fit_transform(X_train)
X_validation_processed = preprocessor.transform(validation_df)

print("Processed training shape:", X_train_processed.shape)
print("Processed validation shape:", X_validation_processed.shape)

In [ ]:
# CHECK FOR NAs:
def has_nan_or_inf(X):
    data = X.data if sparse.issparse(X) else X
    return np.isnan(data).any(), np.isinf(data).any()

print("Train processed:  NA/Inf:", has_nan_or_inf(X_train_processed))
print("Validation processed:  NA/Inf:", has_nan_or_inf(X_validation_processed))

**MLP MODEL:**

Create model:

In [ ]:
# cache to speed up repeated preprocessing
memory = Memory(location=os.path.join(os.getcwd(), "skcache"), verbose=0)

# upgraded MLP: matches baseline's training endurance
tuned_mlp = MLPClassifier(
    max_iter = 200,           # give it room to converge fully
    early_stopping = True,    # same logic, but now consistent patience
    n_iter_no_change = 10,    # same as baseline (no early quit)
    tol = 1e-4,               # stricter tolerance, don't bail too early
    batch_size = 256,         # match baseline batch size
    solver = "adam",
    random_state = 42         # same seed for fair comparison
)

# pipeline: preprocessing + MLP
tuned_pipe = Pipeline([
    ("prep", preprocessor),
    ("mlp", tuned_mlp)
], memory=memory)

# focused but powerful grid
tuned_param_grid = {
    "mlp__hidden_layer_sizes": [(64,), (128,), (128, 64), (96, 48)],
    "mlp__activation": ["relu", "tanh"],
    "mlp__alpha": [1e-4, 1e-3, 1e-2],
    "mlp__learning_rate_init": [5e-4, 1e-3]
    # optional: if you want to let it search endurance knobs too:
    # "mlp__tol": [1e-4, 5e-4],
    # "mlp__n_iter_no_change": [10, 15],
    # "mlp__batch_size": [128, 256],
}

# stronger, more stable CV than the old 3-fold
cv5 = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV with multiple metrics, refit on F1
grid = GridSearchCV(
    estimator = tuned_pipe,
    param_grid = [tuned_param_grid],
    scoring = {
        "acc": "accuracy",
        "f1": "f1",
        "roc": "roc_auc"
    },
    refit = "f1",
    cv = cv5,
    n_jobs = -1,
    verbose = 1
)

Run the model and print insights:

In [ ]:
grid.fit(X_tr, y_tr)


In [ ]:
# top 10 table
cvres = pd.DataFrame(grid.cv_results_).sort_values("rank_test_f1")
cols = ["param_mlp__hidden_layer_sizes","param_mlp__activation","param_mlp__alpha",
        "param_mlp__learning_rate_init","mean_test_acc","mean_test_f1","mean_test_roc","rank_test_f1"]
for c in cols:
    if c not in cvres: cvres[c] = np.nan
display(cvres[cols].head(10))

In [ ]:
# refit on all labeled data, get best pipeline
best_model = grid.best_estimator_

# predictions
y_pred = best_model.predict(X_te)

# ERROR CHECK:
   # if predictions are strings, map to 0 or 1:
y_true_bin = pd.Series(y_te).astype(int).values # y_true is already 0/1
if pd.Series(y_pred).dtype == "O":
    y_pred_bin = (pd.Series(y_pred) == ">50K").astype(int).values
else:                                        
    y_pred_bin = pd.Series(y_pred).astype(int).values

In [ ]:
# get predicted probability for AUC calculation
probs = best_model.predict_proba(X_te)[:, 1]

In [ ]:
# print report
print("Holdout Accuracy:", accuracy_score(y_true_bin, y_pred_bin))
print("Holdout F1:", f1_score(y_true_bin, y_pred_bin))
print("Holdout ROC AUC:", roc_auc_score(y_true_bin, probs))
print("\nClassification Report:\n", classification_report(y_true_bin, y_pred_bin))
print("Confusion Matrix:\n", confusion_matrix(y_true_bin, y_pred_bin))

In [ ]:
# print
print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

In [ ]:
best_model.fit(X_tr, y_tr)

**VISUALIZATIONS:**

In [ ]:
# extract loss curve from the trained model
loss_curve = best_model.named_steps["mlp"].loss_curve_

# plot training loss per epoch
plt.figure(figsize=(8,5))
plt.plot(range(1, len(loss_curve)+1), loss_curve, marker='o', color='purple')
plt.title("MLP Training Loss per Epoch", fontsize=14)
plt.xlabel("Epoch", fontsize=12)
plt.ylabel("Loss", fontsize=12)
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()

MLP Training Loss Curve:
- y-axis shows training loss (model's error)
- x-axis shows epochs (iterations)

Results:
- loss decreases during first few epochs (model quickly picks up on patterns)
- gradual flattening after 10 epochs - training stabilizes, suggests convergence

In [ ]:
# compute confusion matrix
cm = confusion_matrix(y_te, best_model.predict(X_te))

# create and plot display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["<=50K", ">50K"])
disp.plot(cmap="Purples", values_format="d")

# formatting
plt.title("Confusion Matrix – Tuned MLP", fontsize=14)
plt.xlabel("Predicted Label", fontsize=12)
plt.ylabel("True Label", fontsize=12)
plt.grid(False)
plt.tight_layout()
plt.show()

Confusion Matrix:
- shows number of correct/misclassified predictions

Results:
- model achieves approximately 85% accuracy
- correctly predicts most =<50K cases
- correctly predicts about 56-61% of >50K cases
- false negative rate: approx. 44% (meaning around 4 in 10 individuals in >50K class were misclassified)


In [ ]:
RocCurveDisplay.from_estimator(best_model, X_te, y_te)
plt.title("ROC Curve – Tuned MLP")
plt.grid(True, linestyle=":")
plt.show()

ROC Curve:
- shows trade-off between true/false positive rates across thresholds
- y-axis shows how well the model correctly ID >50K earners
- x-axis shows how often the model incorrectly labels <=50K earners as >50K

Results:
- AUC = 0.91, so theres a 91% probability the model correctly gives >50K person a higher score (prob of >50K) than the <=50K person
- model effectively distinguishes between the two groups

In [ ]:
result = permutation_importance(best_model, X_te, y_te, n_repeats=10, random_state=42, n_jobs=-1)

fi_raw = pd.Series(result.importances_mean, index=X_te.columns).sort_values(ascending=True)

plt.figure(figsize=(8,6))
fi_raw.tail(15).plot(kind="barh")
plt.title("Feature Importance – Best MLP")
plt.xlabel("Importance (Mean Decrease in Accuracy)")
plt.tight_layout()
plt.show()

Feature Importance:
- shows what features within the dataset influence predictions most

In [ ]:
# true class distribution
true_counts = y_te.value_counts().rename({0: "<=50K", 1: ">50K"})

# predicted class distribution
pred_counts = pd.Series(best_model.predict(X_te)).value_counts().rename({0: "<=50K", 1: ">50K"})

# combine into one df
compare_df = pd.DataFrame({
    "True": true_counts,
    "Predicted": pred_counts
}).T  # transpose so rows = category type, columns = income classes

# plot
compare_df.plot(
    kind="bar",
    color=["skyblue", "salmon"],
    figsize=(7,5),
    width=0.8
)
plt.title("True vs Predicted Income Distribution")
plt.ylabel("Number of Individuals")
plt.xlabel("Distribution Type")
plt.legend(title="Income Category")
plt.grid(axis="y", linestyle=":", alpha=0.6)
plt.tight_layout()
plt.show()

Predicted Income Distribution: 
Results:
- class imbalance closely resembles the original data

**OTHER MLP ATTEMPT:**

Manual single MLP:

In [ ]:
# base MLP model defined
base_mlp = MLPClassifier(
    max_iter = 150,
    early_stopping = True,      # stop training if validation score doesn't improve
    n_iter_no_change = 10,                # num epochs w no improvement before stopping
    batch_size = 256,           # num training samples used per gradient before weight update
    learning_rate_init = 0.001, # initial learning rate for wight updates
    random_state = 42           # random seed for reproducability
)

base_pipe = Pipeline([
    ("prep", preprocessor),
    ("mlp", base_mlp)
])

In [ ]:
# fit/run model
base_pipe.fit(X_tr, y_tr)

In [ ]:
# predictions from base model
yhat  = base_pipe.predict(X_te) # get final predicted class labels
proba = base_pipe.predict_proba(X_te)[:, 1] # get models confidence for class 1

# print report
print("Holdout accuracy:", accuracy_score(y_te, yhat))
print("Holdout F1:",      f1_score(y_te, yhat))
print("Holdout ROC AUC:", roc_auc_score(y_te, proba))
print("\nClassification Report:\n", classification_report(y_te, yhat))
print("Confusion Matrix:\n", confusion_matrix(y_te, yhat))

In [ ]:
# pull tuned pipeline (already fitted by GridSearchCV on the training folds)
tuned_pipe = best_model         # from: best_model = grid.best_estimator_

**COMPARING BOTH MODELS:**

In [ ]:
# predictions & metrics
y_pred_base  = base_pipe.predict(X_te)
probs_base   = base_pipe.predict_proba(X_te)[:, 1]

y_pred_tuned = tuned_pipe.predict(X_te)
probs_tuned  = tuned_pipe.predict_proba(X_te)[:, 1]

metrics_df = pd.DataFrame({
    "Model":    ["Baseline MLP", "Tuned MLP (GridSearchCV)"],
    "Accuracy": [accuracy_score(y_te, y_pred_base),
                 accuracy_score(y_te, y_pred_tuned)],
    "F1 Score": [f1_score(y_te, y_pred_base),
                 f1_score(y_te, y_pred_tuned)],
    "ROC AUC":  [roc_auc_score(y_te, probs_base),
                 roc_auc_score(y_te, probs_tuned)]
})

# plot
df_melted = metrics_df.melt(id_vars="Model", var_name="Metric", value_name="Score")
plt.figure(figsize=(8,5))
sns.barplot(data=df_melted, x="Metric", y="Score", hue="Model",
            palette=["#b3cde3", "#8856a7"])

for container in plt.gca().containers:
    plt.bar_label(container, fmt="%.3f", label_type="edge", fontsize=10, padding=3)

plt.title("Baseline vs Tuned MLP Performance", fontsize=14)
plt.ylabel("Score")
plt.ylim(0.6, 0.95)
plt.grid(axis="y", linestyle=":", alpha=0.6)
plt.legend(title="Model", loc="lower right")
plt.tight_layout()
plt.show()


Accuracy:
- measures overall proportion of correct predictions across all classes

Results:
- both models are highly accurate (approx. 85%)
- baseline model is identical to tuned model. 

F1 Score:
- shows model performance based on a balance of precision (how accurately >50K cases are predicted) and recall (how many cases of >50K are correctly identified)

Results:
- tuned MLP has a slightly lower F1 score
- indicates that GridSearch tuning didn't improve the baseline model's ability to correctly ID higher income individuals without increasing false positives. With more extensive training and tuning, this would likely change. 

ROC AUC:
- measures how well the model distinguishes between the two classes

Results:
- both are borderline identical, so both models distinguish classes well
- baseline MLP performed .001% better

**ANALYSIS:**

Single MLP:
- trains one NN with fixed settings
- architecture/paramters are manually chosen (hidden layers, activation, etc.)
- only trains once (runs the risk of over or underfitting)
- results are based on one training run
- performance depends on configuration

Tuned MLP with GridSearch:
- explores multiple model configs to find best performer
- automatically tests many combos of hyperparameters
- trains multiple times using cross validation
- best model is selected based on metrics and averaged over folds
- performance is more reliale and generalized bc it's tuned and validated

Comparison:
- accuracy for both was similar (around 85%)
- tuned MLP slightly improved balance between precision and recall shown through F1 score (0.662 vs 0.652) (identifies high income group more effectively without heavily increasing false positives)
- tuned MLP slightly improved ROC AUC, suggesting better class separation

**OUTPUT FILE:**

In [ ]:
# Make copy of best pipeline
best_full = clone(grid.best_estimator_)

# Refit best model on ALL labeled training data
best_full.fit(X_train, y_train)

# Generate predictions for unlabeled validation inputs
val_pred = best_full.predict(validation_df)

# Convert predictions to required format: 1 if >50K, else -1
# First ensure predictions are binary (0 or 1)
if pd.Series(val_pred).dtype == "O":
    # If predictions are strings, convert to binary
    val_pred_bin = (pd.Series(val_pred).str.strip().isin([">50K", ">50K.", ">50k", ">50k."])).astype(int)
else:
    val_pred_bin = pd.Series(val_pred).astype(int)

# Transform: 1 stays as 1 (>50K), 0 becomes -1 (<=50K)
val_pred_transformed = val_pred_bin.replace({0: -1, 1: 1})

# Verify the transformation
print("Value counts of transformed predictions:")
print(val_pred_transformed.value_counts())
print(f"\nTotal predictions: {len(val_pred_transformed)}")

# Save to required CSV file
output_df = pd.DataFrame({"income": val_pred_transformed})
output_df.to_csv("Group_11_MLP_PredictedOutputs.csv", index=False)

print("\n✓ Predictions saved to 'Group_11_MLP_PredictedOutputs.csv'")
print(f"  Format: 1 for >50K, -1 for <=50K")

# Preview first few predictions
print("\nFirst 10 predictions:")
print(output_df.head(10))